In [24]:
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.layers import Dense, Input, Activation, Dropout, Flatten, LeakyReLU, BatchNormalization, Embedding
from keras.layers import concatenate, multiply
from keras.models import Model, Sequential
import pickle
from sklearn.preprocessing import MinMaxScaler

In [25]:
with open("small_signature_dataset.pkl", "rb") as h:
    train_features, test_features, train_labels, test_labels = pickle.load(h)
train_features

array([[-0.627456,  0.215235, -0.990855, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.686103, -0.874932, -1.12914 , ...,  0.      ,  0.      ,
         0.      ],
       [-1.33348 ,  0.73921 ,  0.481883, ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.27783 , -1.69514 ,  0.654642, ...,  0.      ,  0.      ,
         0.      ],
       [ 2.57134 , -1.52128 ,  0.128789, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.23115 ,  1.12001 , -0.594806, ...,  0.      ,  0.      ,
         0.      ]])

In [26]:
train_labels

array([-4,  2,  0, ...,  2, -2,  0], dtype=int64)

In [27]:
fixed_labels = (train_labels + 4)/2

In [28]:
latent_dim = 100
len_dataset = train_features.shape[0]
len_features = train_features.shape[1]
n_classes = len(set(list(train_labels)))

In [29]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_features = scaler.fit_transform(train_features)

In [30]:
def build_discriminator(optimizer=Adam(0.0002, 0.5)):
    # Inputs
    features = Input(shape=(len_features,))
    label = Input(shape=(1,), dtype="int32")
    
    # Embedding layer
    label_embedding = Flatten()(Embedding(n_classes, len_features)(label))
    
    # Condition discrimination of generated features
    inputs = multiply([features, label_embedding])
    
    # Dense layers
    x = Dense(512)(inputs)
    x = LeakyReLU(alpha=.2)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=.2)(x)
    x = Dropout(.4)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=.2)(x)
    x = Dropout(.4)(x)
    
    output = Dense(1, activation="sigmoid")(x)
    
    model = Model([features, label], output)
    model.compile(loss="binary_crossentropy",
                 optimizer=optimizer,
                 metrics=["accuracy"])
    model.summary()

    return model

In [31]:
discriminator = build_discriminator()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 1, 41)        205         ['input_8[0][0]']                
                                                                                                  
 input_7 (InputLayer)           [(None, 41)]         0           []                               
                                                                                                  
 flatten_2 (Flatten)            (None, 41)           0           ['embedding_3[0][0]']            
                                                                                            

In [32]:
def build_generator():
    # Inputs
    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype="int32")
    
    # Embedding layer
    label_embedding = Flatten()(Embedding(n_classes, latent_dim)(label))
    
    # Condition generation of features
    inputs = multiply([noise, label_embedding])
    
    x = Dense(256)(inputs)
    x = LeakyReLU(alpha=.2)(x)
    x = BatchNormalization(momentum=.8)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=.2)(x)
    x = BatchNormalization(momentum=.8)(x)
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=.2)(x)
    x = BatchNormalization(momentum=.8)(x)
    
    output = Dense(len_features, activation="tanh")(x)
    
    model = Model([noise, label], output)
    model.summary()
    
    return model

In [33]:
Embedding(n_classes, latent_dim)(tf.constant([1])).numpy()

array([[-0.03050144,  0.01369567, -0.01331947, -0.00803236, -0.03913509,
        -0.01830154,  0.00344079, -0.01072671,  0.04661864, -0.00305668,
        -0.00292891,  0.02457938,  0.01306815,  0.02203229,  0.01204081,
         0.01973106,  0.0194339 , -0.01816284, -0.03164823,  0.00897754,
        -0.04968693,  0.00378522,  0.01534916,  0.03953044,  0.02012945,
        -0.01211605, -0.01918317,  0.02874532, -0.02834987,  0.04285104,
         0.00627499,  0.02507887,  0.0379361 , -0.01193865, -0.00425778,
         0.00152149,  0.02129911,  0.00742546, -0.04393261,  0.01683149,
         0.03787014, -0.0330001 ,  0.00568663,  0.00919882, -0.00460963,
         0.00037625, -0.00243304, -0.01163309,  0.03077003,  0.03216041,
         0.01231784, -0.02601285, -0.04448634, -0.01313506,  0.04042426,
         0.00759789,  0.04118121, -0.04270251, -0.03732804, -0.03458756,
         0.0218107 ,  0.01679485,  0.02481589,  0.00298406, -0.03264578,
         0.04016152, -0.03867345, -0.04393342,  0.0

In [34]:
generator = build_generator()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 1, 100)       500         ['input_10[0][0]']               
                                                                                                  
 input_9 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 flatten_3 (Flatten)            (None, 100)          0           ['embedding_5[0][0]']            
                                                                                            

In [35]:
def build_cgan(generator, discriminator, optimizer=Adam(.0002, .5)):
    # Inputs
    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,))
    
    # Where the generator and discriminator come in
    features = generator([noise, label])
    valid = discriminator([features, label])
    
    # Discriminator is trained separately
    discriminator.trainable = False
    
    model = Model([noise, label], valid)
    model.compile(loss="binary_crossentropy",
                 optimizer=optimizer,
                 metrics=["accuracy"])
    model.summary()

    return model

In [36]:
cgan = build_cgan(generator, discriminator)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 model_4 (Functional)           (None, 41)           732445      ['input_11[0][0]',               
                                                                  'input_12[0][0]']               
                                                                                                  
 model_3 (Functional)           (None, 1)            547534      ['model_4[0][0]',          

In [37]:
def get_random_batch(X, y, batch_size):
    '''
    Will return random batches of size batch_size
    
    Params:
        X: numpy array - features
        y: numpy array - classes
        batch_size: Int
    '''
    idx = np.random.randint(0, len(X))
    
    X_batch = X[idx:idx+batch_size]
    y_batch = y[idx:idx+batch_size]
    
    return X_batch, y_batch

In [38]:
def train_gan(cgan, generator, discriminator,
             X, y,
             n_epochs=10000, batch_size=32,
             hist_every=100, log_every=1000):
    half_batch = int(batch_size/2)
    
    acc_real_hist = []
    acc_fake_hist = []
    acc_gan_hist = []
    loss_real_hist = []
    loss_fake_hist = []
    loss_gan_hist = []
    
    for epoch in range(n_epochs):
        
        X_batch, labels = get_random_batch(X, y, batch_size)
        
        # train with real values
        y_real = np.ones((X_batch.shape[0], 1))
        loss_real, acc_real = discriminator.train_on_batch([X_batch, labels], y_real)
        
        # train with fake values
        noise = np.random.uniform(0, 1, (labels.shape[0], latent_dim))
        X_fake = generator.predict([noise, labels])
        y_fake = np.zeros((X_fake.shape[0], 1))
        loss_fake, acc_fake = discriminator.train_on_batch([X_fake, labels], y_fake)
        
        y_gan = np.ones((labels.shape[0], 1))
        loss_gan, acc_gan = cgan.train_on_batch([noise, labels], y_gan)
        
        if (epoch + 1) % hist_every == 0:
            acc_real_hist.append(acc_real)
            acc_fake_hist.append(acc_fake)
            acc_gan_hist.append(acc_gan)
            loss_real_hist.append(loss_real)
            loss_fake_hist.append(loss_fake)
            loss_gan_hist.append(loss_gan)
        
        if (epoch + 1) % log_every == 0:
            lr = "loss real: {:.3f}".format(loss_real)
            ar = "acc real: {:.3f}".format(acc_real)
            lf = "loss fake: {:.3f}".format(loss_fake)
            af = "acc fake: {:.3f}".format(acc_fake)
            lg = "loss gan: {:.3f}".format(loss_gan)
            ag = "acc gan: {:.3f}".format(acc_gan)
            
            print("{}, {} | {}, {} | {}, {}".format(lr, ar, lf, af, lg, ag))
            
        print("Complet {} epochs out of {}".format(epoch + 1, n_epochs))
        
    return loss_real_hist, acc_real_hist, loss_fake_hist, acc_fake_hist, loss_gan_hist, acc_gan_hist

In [39]:
train_gan(cgan, generator, discriminator, scaled_features, fixed_labels)

Complet 1 epochs out of 10000
Complet 2 epochs out of 10000
Complet 3 epochs out of 10000
Complet 4 epochs out of 10000
Complet 5 epochs out of 10000
Complet 6 epochs out of 10000
Complet 7 epochs out of 10000
Complet 8 epochs out of 10000
Complet 9 epochs out of 10000
Complet 10 epochs out of 10000
Complet 11 epochs out of 10000
Complet 12 epochs out of 10000
Complet 13 epochs out of 10000
Complet 14 epochs out of 10000
Complet 15 epochs out of 10000
Complet 16 epochs out of 10000
Complet 17 epochs out of 10000
Complet 18 epochs out of 10000
Complet 19 epochs out of 10000
Complet 20 epochs out of 10000
Complet 21 epochs out of 10000
Complet 22 epochs out of 10000
Complet 23 epochs out of 10000
Complet 24 epochs out of 10000
Complet 25 epochs out of 10000
Complet 26 epochs out of 10000
Complet 27 epochs out of 10000
Complet 28 epochs out of 10000
Complet 29 epochs out of 10000
Complet 30 epochs out of 10000
Complet 31 epochs out of 10000
Complet 32 epochs out of 10000
Complet 33 epochs

Complet 261 epochs out of 10000
Complet 262 epochs out of 10000
Complet 263 epochs out of 10000
Complet 264 epochs out of 10000
Complet 265 epochs out of 10000
Complet 266 epochs out of 10000
Complet 267 epochs out of 10000
Complet 268 epochs out of 10000
Complet 269 epochs out of 10000
Complet 270 epochs out of 10000
Complet 271 epochs out of 10000
Complet 272 epochs out of 10000
Complet 273 epochs out of 10000
Complet 274 epochs out of 10000
Complet 275 epochs out of 10000
Complet 276 epochs out of 10000
Complet 277 epochs out of 10000
Complet 278 epochs out of 10000
Complet 279 epochs out of 10000
Complet 280 epochs out of 10000
Complet 281 epochs out of 10000
Complet 282 epochs out of 10000
Complet 283 epochs out of 10000
Complet 284 epochs out of 10000
Complet 285 epochs out of 10000
Complet 286 epochs out of 10000
Complet 287 epochs out of 10000
Complet 288 epochs out of 10000
Complet 289 epochs out of 10000
Complet 290 epochs out of 10000
Complet 291 epochs out of 10000
Complet 

Complet 519 epochs out of 10000
Complet 520 epochs out of 10000
Complet 521 epochs out of 10000
Complet 522 epochs out of 10000
Complet 523 epochs out of 10000
Complet 524 epochs out of 10000
Complet 525 epochs out of 10000
Complet 526 epochs out of 10000
Complet 527 epochs out of 10000
Complet 528 epochs out of 10000
Complet 529 epochs out of 10000
Complet 530 epochs out of 10000
Complet 531 epochs out of 10000
Complet 532 epochs out of 10000
Complet 533 epochs out of 10000
Complet 534 epochs out of 10000
Complet 535 epochs out of 10000
Complet 536 epochs out of 10000
Complet 537 epochs out of 10000
Complet 538 epochs out of 10000
Complet 539 epochs out of 10000
Complet 540 epochs out of 10000
Complet 541 epochs out of 10000
Complet 542 epochs out of 10000
Complet 543 epochs out of 10000
Complet 544 epochs out of 10000
Complet 545 epochs out of 10000
Complet 546 epochs out of 10000
Complet 547 epochs out of 10000
Complet 548 epochs out of 10000
Complet 549 epochs out of 10000
Complet 

Complet 777 epochs out of 10000
Complet 778 epochs out of 10000
Complet 779 epochs out of 10000
Complet 780 epochs out of 10000
Complet 781 epochs out of 10000
Complet 782 epochs out of 10000
Complet 783 epochs out of 10000
Complet 784 epochs out of 10000
Complet 785 epochs out of 10000
Complet 786 epochs out of 10000
Complet 787 epochs out of 10000
Complet 788 epochs out of 10000
Complet 789 epochs out of 10000
Complet 790 epochs out of 10000
Complet 791 epochs out of 10000
Complet 792 epochs out of 10000
Complet 793 epochs out of 10000
Complet 794 epochs out of 10000
Complet 795 epochs out of 10000
Complet 796 epochs out of 10000
Complet 797 epochs out of 10000
Complet 798 epochs out of 10000
Complet 799 epochs out of 10000
Complet 800 epochs out of 10000
Complet 801 epochs out of 10000
Complet 802 epochs out of 10000
Complet 803 epochs out of 10000
Complet 804 epochs out of 10000
Complet 805 epochs out of 10000
Complet 806 epochs out of 10000
Complet 807 epochs out of 10000
Complet 

Complet 1030 epochs out of 10000
Complet 1031 epochs out of 10000
Complet 1032 epochs out of 10000
Complet 1033 epochs out of 10000
Complet 1034 epochs out of 10000
Complet 1035 epochs out of 10000
Complet 1036 epochs out of 10000
Complet 1037 epochs out of 10000
Complet 1038 epochs out of 10000
Complet 1039 epochs out of 10000
Complet 1040 epochs out of 10000
Complet 1041 epochs out of 10000
Complet 1042 epochs out of 10000
Complet 1043 epochs out of 10000
Complet 1044 epochs out of 10000
Complet 1045 epochs out of 10000
Complet 1046 epochs out of 10000
Complet 1047 epochs out of 10000
Complet 1048 epochs out of 10000
Complet 1049 epochs out of 10000
Complet 1050 epochs out of 10000
Complet 1051 epochs out of 10000
Complet 1052 epochs out of 10000
Complet 1053 epochs out of 10000
Complet 1054 epochs out of 10000
Complet 1055 epochs out of 10000
Complet 1056 epochs out of 10000
Complet 1057 epochs out of 10000
Complet 1058 epochs out of 10000
Complet 1059 epochs out of 10000
Complet 10

...

Complet 9242 epochs out of 10000
Complet 9243 epochs out of 10000
Complet 9244 epochs out of 10000
Complet 9245 epochs out of 10000
Complet 9246 epochs out of 10000
Complet 9247 epochs out of 10000
Complet 9248 epochs out of 10000
Complet 9249 epochs out of 10000
Complet 9250 epochs out of 10000
Complet 9251 epochs out of 10000
Complet 9252 epochs out of 10000
Complet 9253 epochs out of 10000
Complet 9254 epochs out of 10000
Complet 9255 epochs out of 10000
Complet 9256 epochs out of 10000
Complet 9257 epochs out of 10000
Complet 9258 epochs out of 10000
Complet 9259 epochs out of 10000
Complet 9260 epochs out of 10000
Complet 9261 epochs out of 10000
Complet 9262 epochs out of 10000
Complet 9263 epochs out of 10000
Complet 9264 epochs out of 10000
Complet 9265 epochs out of 10000
Complet 9266 epochs out of 10000
Complet 9267 epochs out of 10000
Complet 9268 epochs out of 10000
Complet 9269 epochs out of 10000
Complet 9270 epochs out of 10000
Complet 9271 epochs out of 10000
Complet 92

Complet 9491 epochs out of 10000
Complet 9492 epochs out of 10000
Complet 9493 epochs out of 10000
Complet 9494 epochs out of 10000
Complet 9495 epochs out of 10000
Complet 9496 epochs out of 10000
Complet 9497 epochs out of 10000
Complet 9498 epochs out of 10000
Complet 9499 epochs out of 10000
Complet 9500 epochs out of 10000
Complet 9501 epochs out of 10000
Complet 9502 epochs out of 10000
Complet 9503 epochs out of 10000
Complet 9504 epochs out of 10000
Complet 9505 epochs out of 10000
Complet 9506 epochs out of 10000
Complet 9507 epochs out of 10000
Complet 9508 epochs out of 10000
Complet 9509 epochs out of 10000
Complet 9510 epochs out of 10000
Complet 9511 epochs out of 10000
Complet 9512 epochs out of 10000
Complet 9513 epochs out of 10000
Complet 9514 epochs out of 10000
Complet 9515 epochs out of 10000
Complet 9516 epochs out of 10000
Complet 9517 epochs out of 10000
Complet 9518 epochs out of 10000
Complet 9519 epochs out of 10000
Complet 9520 epochs out of 10000
Complet 95

Complet 9740 epochs out of 10000
Complet 9741 epochs out of 10000
Complet 9742 epochs out of 10000
Complet 9743 epochs out of 10000
Complet 9744 epochs out of 10000
Complet 9745 epochs out of 10000
Complet 9746 epochs out of 10000
Complet 9747 epochs out of 10000
Complet 9748 epochs out of 10000
Complet 9749 epochs out of 10000
Complet 9750 epochs out of 10000
Complet 9751 epochs out of 10000
Complet 9752 epochs out of 10000
Complet 9753 epochs out of 10000
Complet 9754 epochs out of 10000
Complet 9755 epochs out of 10000
Complet 9756 epochs out of 10000
Complet 9757 epochs out of 10000
Complet 9758 epochs out of 10000
Complet 9759 epochs out of 10000
Complet 9760 epochs out of 10000
Complet 9761 epochs out of 10000
Complet 9762 epochs out of 10000
Complet 9763 epochs out of 10000
Complet 9764 epochs out of 10000
Complet 9765 epochs out of 10000
Complet 9766 epochs out of 10000
Complet 9767 epochs out of 10000
Complet 9768 epochs out of 10000
Complet 9769 epochs out of 10000
Complet 97

Complet 9990 epochs out of 10000
Complet 9991 epochs out of 10000
Complet 9992 epochs out of 10000
Complet 9993 epochs out of 10000
Complet 9994 epochs out of 10000
Complet 9995 epochs out of 10000
Complet 9996 epochs out of 10000
Complet 9997 epochs out of 10000
Complet 9998 epochs out of 10000
Complet 9999 epochs out of 10000
loss real: 0.000, acc real: 1.000 | loss fake: 0.146, acc fake: 0.969 | loss gan: 4.377, acc gan: 0.000
Complet 10000 epochs out of 10000


([0.0638420432806015,
  0.41339486837387085,
  0.16262966394424438,
  0.6200740337371826,
  0.17310784757137299,
  0.2544505298137665,
  0.2524826228618622,
  0.2064221203327179,
  0.1748390793800354,
  0.1957148015499115,
  0.4465876817703247,
  0.24423815310001373,
  0.06616471707820892,
  0.657860279083252,
  0.25143420696258545,
  0.2634672224521637,
  0.38366344571113586,
  0.7154910564422607,
  0.2605048418045044,
  0.2184167504310608,
  0.21202094852924347,
  0.1931568682193756,
  0.26111817359924316,
  0.0384112186729908,
  0.4210396707057953,
  0.5102357864379883,
  0.41604113578796387,
  0.05289802327752113,
  0.02460462599992752,
  0.07538685202598572,
  1.012587547302246,
  0.026063837110996246,
  0.021115802228450775,
  0.19793438911437988,
  0.14978240430355072,
  0.20770052075386047,
  0.026555940508842468,
  0.05201048403978348,
  0.1345575898885727,
  0.00830017775297165,
  0.043958913534879684,
  0.17011643946170807,
  0.14074349403381348,
  0.0013784768525511026,
  0

In [40]:
def rev_min_max_func(scaled_val):
    max_val = max(train_features.flatten())
    min_val = min(train_features.flatten())
    og_val = (scaled_val*(max_val - min_val)) + min_val
    return og_val

In [43]:
def generate_samples(class_for, n_samples=20):
    '''
    Generates new random but very realistic features using
    a trained generator model
    
    Params:
        class_for: Int - features for this class
        n_samples: Int - how many samples to generate
    '''
    
    noise = np.random.uniform(0, 1, (n_samples, latent_dim))
    label = np.full((n_samples,), fill_value=class_for)
    return rev_min_max_func(generator.predict([noise, label]))

In [47]:
for i in range(5):
    print("Here are 20 samples of generated permutations with signature {}".format(2*(i - 2)))
    print(generate_samples(i))

Here are 20 samples of generated permutations with signature -4
[[ 3.59888601e+00 -1.51127930e+01 -1.51351185e+01  3.92839766e+00
   3.98856878e+00  2.67690563e+00 -1.42993469e+01  4.58344460e-01
  -6.98357487e+00 -1.26520853e+01 -1.48876820e+01 -6.90368605e+00
  -3.60094643e+00  3.73743773e+00 -2.58649850e+00  3.57476282e+00
  -1.47890644e+01 -2.32149291e+00  2.55571508e+00 -1.47882957e+01
  -1.13282242e+01 -1.34954987e+01 -1.16062241e+01  2.74899244e+00
  -1.77440834e+00 -1.42091007e+01  2.61228800e+00 -8.16200733e+00
  -1.47986469e+01  3.30353308e+00 -9.15064144e+00 -1.07661858e+01
   2.03347969e+00 -1.42764397e+01 -1.08735352e+01 -1.41249046e+01
  -7.04111814e+00 -5.52950096e+00 -1.87740588e+00 -3.64545822e-01
  -5.11255789e+00]
 [ 3.45009470e+00 -1.50818939e+01 -1.51146297e+01  3.66046381e+00
   3.81829023e+00  2.30164242e+00 -1.42729683e+01  6.97139263e-01
  -6.76309681e+00 -1.20958977e+01 -1.47499523e+01 -6.68679237e+00
  -3.28285217e+00  3.73158216e+00 -2.45736408e+00  3.372626

[[  2.8980498    3.8879514    3.7856202  -12.829742   -14.28812
  -13.603968     4.4181542    4.0880284   -3.1043155    3.1755157
    3.7530017    2.959682   -13.038119     2.849362   -14.4437065
  -14.803911     1.23176    -13.159231   -10.562196   -13.884127
  -14.899012   -12.990921     2.3952427  -14.233982   -13.633408
  -14.741276   -11.690346   -13.393219    -8.311214     4.0264573
  -10.193998   -14.298664     2.2334943    1.866056     3.8729348
   -4.9553914   -5.08149    -10.493372    -5.391094    -2.3050237
   -4.675915  ]
 [ -1.9055924   -2.2233863   -2.2662208   -9.903017    -9.303163
   -5.126992   -13.275709     0.8832221   -5.7762623   -6.1613355
    1.7209558   -4.8778973    1.4407296    3.1208797    1.8920164
   -2.2161026  -13.285299    -2.177746   -13.519005    -4.5186834
   -7.9033785  -11.498317    -1.4143465  -11.385005   -10.885462
  -11.914278    -0.9227276   -7.0429125  -11.56429     -0.21748829
   -3.3471875   -1.0996604   -5.940475    -7.037353    -4.232201


[[-9.64481449e+00  1.69899464e+00  3.93452978e+00 -3.49890995e+00
  -8.75058365e+00  2.09758377e+00 -9.50797749e+00 -1.10924454e+01
  -1.07194958e+01 -1.35984383e+01 -4.91274071e+00 -1.39705639e+01
   3.84181738e+00 -1.28464012e+01  4.15722513e+00  1.48619366e+00
  -1.04055452e+00 -2.69989586e+00 -1.27230434e+01  1.86015701e+00
   2.31756544e+00 -1.35451221e+01 -8.86536503e+00 -5.83049107e+00
  -5.49721766e+00  1.38033104e+00 -7.42186594e+00 -4.41972113e+00
  -3.48292542e+00 -9.18983078e+00 -9.94686985e+00  2.43614578e+00
  -1.07340736e+01 -1.33690453e+01 -1.45638676e+01 -4.48799896e+00
  -1.51234484e+00 -2.68152452e+00 -4.99824238e+00  9.55855370e-01
  -4.93652582e+00]
 [-2.52830410e+00 -2.66652155e+00 -2.53798842e+00 -9.46329498e+00
  -9.51290894e+00 -4.77083683e+00 -1.35739632e+01  4.43520546e-01
  -6.02852106e+00 -6.52240801e+00  1.61224937e+00 -5.49252605e+00
   1.76221228e+00  3.03051233e+00  2.17530251e+00 -1.57354999e+00
  -1.32735205e+01 -2.07786918e+00 -1.35602779e+01 -3.6729

[[ -3.3358285   -4.0911636   -4.308977    -9.024722    -8.549332
   -4.976301   -14.068848     0.10061646  -5.6566167   -5.9100313
    1.0734863   -5.698399     1.5500565    3.0805488    2.037341
   -1.238966   -13.477989    -1.6980321  -13.277493    -3.1279547
   -6.0677824  -10.324619    -1.8586328  -10.437031    -9.642182
  -10.930477    -0.25293636  -5.201106   -11.091545    -2.0613065
   -3.050708    -0.48364067  -6.650199    -7.0430274   -4.750893
   -5.508212    -4.280052    -5.7138433   -2.5944793   -0.43958092
   -5.337915  ]
 [-14.719429   -15.102602   -15.178764   -11.601978     4.2840333
  -13.3621235  -15.157427   -14.187071     1.8614502    1.8558936
  -13.163802     2.7641435  -14.51226     -0.76999617 -14.603401
    0.84489536 -15.026182     3.786264     4.149511     1.8899231
    4.143169     4.5797744   -7.42167      2.9926963    3.0292583
    3.5355725    3.6606555    3.485293     2.1353364  -15.072056
    4.5038238    2.608354   -12.192964     3.8508277    4.0188947

[[-2.85771608e+00 -3.08012891e+00 -2.55318236e+00 -8.94766045e+00
  -9.59370327e+00 -4.04845238e+00 -1.36932507e+01 -5.80377579e-02
  -6.30174160e+00 -7.11312008e+00  1.29627752e+00 -6.27112389e+00
   1.94973326e+00  2.83150053e+00  2.47322130e+00 -1.02128506e+00
  -1.30985050e+01 -2.01738000e+00 -1.34571419e+01 -3.31406879e+00
  -6.06650305e+00 -1.14814596e+01 -2.30886960e+00 -1.04811649e+01
  -9.75420952e+00 -1.09546185e+01 -8.93017769e-01 -6.01994753e+00
  -1.15398331e+01 -1.29039907e+00 -3.80430317e+00 -2.01467037e-01
  -6.94965076e+00 -8.03550720e+00 -5.76373529e+00 -5.69922638e+00
  -3.94482183e+00 -5.45540333e+00 -2.95165324e+00 -4.04906273e-01
  -5.20478296e+00]
 [-2.96892333e+00 -2.90801454e+00 -2.34964442e+00 -9.06265163e+00
  -9.60601807e+00 -4.03674507e+00 -1.36876030e+01 -6.66227341e-02
  -6.38267231e+00 -7.06862640e+00  1.35524273e+00 -6.22255898e+00
   1.96322012e+00  2.79625940e+00  2.45970058e+00 -1.08351898e+00
  -1.30814676e+01 -1.98747301e+00 -1.34822826e+01 -3.2053